In [1]:
from rdkit import Chem
import pyaniasetools as pya

from ase_interface import ANIENS,ensemblemolecule
import hdnntools as hdt
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

np.set_printoptions(threshold=1000000000,suppress=True)

van Der Waals correction will be unavailable. Please install ased3


In [2]:
ntdir = '/nh/nest/u/jsmith/scratch/Gits/ANI-Networks/networks/al_networks/ANI-AL-0808.0303.0400/'
cns = ntdir + 'train0/rHCNOSFCl-4.6A_16-3.1A_a4-8.params'
sae = ntdir + 'train0/sae_wb97x-631gd.dat'
nnf = ntdir + 'train'

#ntdir = '/nh/nest/u/jsmith/scratch/Research/ccsd_extrapolation/new_ccsd_data/ccsd_train/'
#cns = ntdir + 'rHCNO-4.6A_16-3.1A_a4-8.params'
#sae = ntdir + 'sae_linfit.dat'
#nnf = ntdir + 'train'

datai = "/nh/nest/u/jsmith/Research/Data_Oles/drugbank_3d_1564.sdf"
datao = "/nh/nest/u/jsmith/Research/Data_Oles/ani_data.sdf"

Nn = 5

In [3]:
# Set ANI calculator
aec = pya.anienscomputetool(cns, sae, nnf, Nn, 0)

In [4]:
suppl = Chem.SDMolSupplier(datai, removeHs=False)

In [5]:
writer = Chem.SDWriter(datao)
for i,mol in enumerate(suppl):
    X, S = pya.__convert_rdkitmol_to_nparr__(mol)
    if (set(S).issubset(['C', 'H', 'N', 'O', 'S', 'F', 'Cl'])):
        print("Molecule:",i,mol.GetNumAtoms(),set(S))
        opt = aec.optimize_rdkit_molecule(mol,-1)
        E,sig = aec.energy_rdkit_conformers(mol,[0])
        
        mol.SetProp("ANIenergy",str(E))
        mol.SetProp("ANIstddev",str(sig))
        
        activations = dict({"layer0":[],"layer1":[],"layer2":[],})
        for i in range(3):
            for j in range(mol.GetNumAtoms()):
                activations["layer"+str(i)].append(aec.ens.ncl[0].activations(j,i,0))
            activations["layer"+str(i)] = np.vstack(activations["layer"+str(i)])
            
        for k in activations.keys():
            mol.SetProp(k,np.array2string(activations[k]))
        
        writer.write(mol)

Molecule: 0 55 {'H', 'N', 'C', 'O'}
Molecule: 1 20 {'H', 'N', 'C', 'O'}
Molecule: 2 20 {'H', 'N', 'C', 'O'}
Molecule: 3 10 {'H', 'C', 'O'}
Molecule: 4 23 {'H', 'N', 'C', 'O'}
Molecule: 5 32 {'O', 'N', 'S', 'C', 'H'}
Molecule: 6 24 {'H', 'N', 'C', 'O'}
Molecule: 7 26 {'H', 'N', 'C', 'O'}
Molecule: 8 20 {'H', 'C', 'O'}
Molecule: 9 40 {'H', 'N', 'C'}
Molecule: 10 16 {'H', 'N', 'C', 'O'}
Molecule: 11 21 {'H', 'N', 'C', 'O'}
Molecule: 12 20 {'H', 'N', 'C', 'O'}
Molecule: 14 50 {'H', 'C', 'O'}
Molecule: 15 14 {'H', 'N', 'C', 'O'}
Molecule: 16 20 {'O', 'N', 'S', 'C', 'H'}
Molecule: 17 24 {'H', 'N', 'C', 'O'}
Molecule: 18 74 {'H', 'C', 'O'}
Molecule: 19 98 {'H', 'C', 'O'}
Molecule: 20 26 {'O', 'N', 'S', 'C', 'H'}
Molecule: 21 14 {'H', 'C', 'O'}
Molecule: 22 47 {'H', 'N', 'C', 'O'}
Molecule: 23 30 {'H', 'N', 'C', 'O'}
Molecule: 24 37 {'O', 'N', 'S', 'C', 'H'}
Molecule: 26 10 {'H', 'N', 'C', 'O'}
Molecule: 27 73 {'H', 'C', 'O'}
Molecule: 28 21 {'H', 'N', 'C', 'O'}
Molecule: 29 18 {'H', 'N', 'C',

Molecule: 226 87 {'O', 'N', 'C', 'F', 'H'}
Molecule: 227 50 {'H', 'N', 'C', 'O'}
Molecule: 228 25 {'H', 'N', 'C', 'O'}
Molecule: 229 22 {'O', 'N', 'S', 'C', 'H'}
Molecule: 230 119 {'H', 'C', 'O'}
Molecule: 231 46 {'O', 'N', 'S', 'C', 'H'}
Molecule: 232 46 {'H', 'N', 'S', 'C'}
Molecule: 233 56 {'H', 'N', 'C', 'O'}
Molecule: 234 42 {'H', 'N', 'C', 'O'}
Molecule: 235 53 {'H', 'C', 'O'}
Molecule: 236 24 {'H', 'N', 'C', 'O'}
Molecule: 237 48 {'O', 'N', 'C', 'F', 'Cl', 'H'}
Molecule: 239 44 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 240 35 {'H', 'Cl', 'N', 'C'}
Molecule: 242 41 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 244 79 {'H', 'C', 'O'}
Molecule: 245 25 {'H', 'N', 'C', 'O'}
Molecule: 246 44 {'O', 'N', 'S', 'C', 'H'}
Molecule: 247 31 {'H', 'N', 'S', 'C'}
Molecule: 248 43 {'O', 'N', 'S', 'C', 'H'}
Molecule: 249 42 {'O', 'N', 'S', 'C', 'H'}
Molecule: 250 35 {'H', 'N', 'C', 'O'}
Molecule: 251 36 {'H', 'N', 'C', 'O'}
Molecule: 252 40 {'H', 'N', 'S', 'C'}
Molecule: 253 36 {'H', 'N', 'C', 'O'}
Molecule: 

Molecule: 444 45 {'O', 'N', 'S', 'C', 'H'}
Molecule: 445 30 {'O', 'N', 'S', 'C', 'Cl', 'H'}
Molecule: 446 58 {'O', 'N', 'C', 'F', 'H'}
Molecule: 447 58 {'O', 'N', 'C', 'F', 'H'}
Molecule: 448 57 {'H', 'N', 'C', 'O'}
Molecule: 449 28 {'H', 'C', 'O'}
Molecule: 451 53 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 452 53 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 453 53 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 454 53 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 455 51 {'H', 'N', 'S', 'C'}
Molecule: 456 55 {'O', 'N', 'S', 'C', 'H'}
Molecule: 457 138 {'H', 'N', 'C', 'O'}
Molecule: 458 39 {'H', 'C', 'O'}
Molecule: 459 36 {'N', 'C', 'F', 'Cl', 'H'}
Molecule: 460 69 {'H', 'N', 'C', 'O'}
Molecule: 461 45 {'O', 'N', 'C', 'F', 'H'}
Molecule: 462 62 {'H', 'N', 'C', 'O'}
Molecule: 463 47 {'O', 'N', 'S', 'C', 'H'}
Molecule: 464 50 {'O', 'N', 'C', 'F', 'Cl', 'H'}
Molecule: 465 70 {'H', 'N', 'C', 'O'}
Molecule: 466 40 {'H', 'N', 'C', 'O'}
Molecule: 467 40 {'H', 'N', 'C', 'O'}
Molecule: 468 37 {'H', 'C', 'O'}
Molecule: 469 67 {'H

Molecule: 662 33 {'H', 'N', 'C', 'O'}
Molecule: 663 56 {'O', 'N', 'S', 'C', 'H'}
Molecule: 664 42 {'H', 'N', 'C'}
Molecule: 665 43 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 666 35 {'O', 'N', 'S', 'C', 'H'}
Molecule: 667 29 {'H', 'N', 'C', 'O'}
Molecule: 668 65 {'H', 'C', 'O'}
Molecule: 669 54 {'H', 'N', 'C', 'O'}
Molecule: 670 75 {'O', 'N', 'S', 'C', 'F', 'H'}
Molecule: 671 52 {'O', 'N', 'S', 'C', 'H'}
Molecule: 672 43 {'H', 'N', 'C', 'O'}
Molecule: 673 43 {'H', 'N', 'C', 'O'}
Molecule: 674 16 {'H', 'C', 'O'}
Molecule: 675 34 {'H', 'N', 'C', 'O'}
Molecule: 676 67 {'H', 'N', 'C', 'O'}
Molecule: 677 30 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 678 50 {'H', 'N', 'C', 'O'}
Molecule: 679 28 {'H', 'N', 'C', 'O'}
Molecule: 680 21 {'H', 'C', 'O'}
Molecule: 681 37 {'H', 'C', 'O'}
Molecule: 682 88 {'O', 'S', 'C', 'F', 'H'}
Molecule: 683 61 {'O', 'N', 'S', 'C', 'H'}
Molecule: 684 31 {'H', 'N', 'C', 'O'}
Molecule: 685 76 {'H', 'N', 'C', 'O'}
Molecule: 686 17 {'H', 'N', 'C', 'O'}
Molecule: 687 48 {'O', 'N', 

Molecule: 876 63 {'H', 'N', 'C', 'O'}
Molecule: 877 29 {'O', 'N', 'S', 'C', 'Cl', 'H'}
Molecule: 878 52 {'H', 'C', 'O'}
Molecule: 879 88 {'H', 'N', 'C', 'O'}
Molecule: 881 52 {'H', 'N', 'C', 'O'}
Molecule: 882 54 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 884 32 {'H', 'N', 'C', 'O'}
Molecule: 885 32 {'H', 'N', 'C', 'O'}
Molecule: 886 12 {'H', 'F', 'C', 'O'}
Molecule: 887 60 {'O', 'N', 'S', 'C', 'Cl', 'H'}
Molecule: 888 32 {'H', 'F', 'N', 'C'}
Molecule: 889 52 {'H', 'N', 'C', 'O'}
Molecule: 890 44 {'O', 'N', 'S', 'C', 'H'}
Molecule: 891 48 {'O', 'N', 'C', 'F', 'H'}
Molecule: 892 60 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 893 29 {'O', 'N', 'S', 'C', 'H'}
Molecule: 894 26 {'H', 'N', 'C', 'O'}
Molecule: 895 49 {'H', 'N', 'C', 'O'}
Molecule: 896 60 {'H', 'N', 'C', 'O'}
Molecule: 897 36 {'O', 'N', 'C', 'F', 'H'}
Molecule: 898 31 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 899 43 {'O', 'N', 'C', 'F', 'H'}
Molecule: 900 50 {'O', 'N', 'C', 'F', 'H'}
Molecule: 901 46 {'H', 'N', 'C', 'O'}
Molecule: 902 54 {'O', 

Molecule: 1088 15 {'H', 'N', 'C', 'O'}
Molecule: 1089 28 {'H', 'N', 'C', 'O'}
Molecule: 1091 14 {'H', 'N', 'C', 'O'}
Molecule: 1092 13 {'H', 'C', 'O'}
Molecule: 1093 65 {'H', 'N', 'C', 'O'}
Molecule: 1094 68 {'H', 'C', 'O'}
Molecule: 1095 19 {'H', 'N', 'C', 'O'}
Molecule: 1096 15 {'H', 'C', 'O'}
Molecule: 1097 8 {'H', 'N', 'C', 'O'}
Molecule: 1098 54 {'H', 'C', 'O'}
Molecule: 1099 25 {'H', 'C', 'O'}
Molecule: 1100 20 {'O', 'N', 'S', 'C', 'H'}
Molecule: 1101 12 {'H', 'C', 'O'}
Molecule: 1102 45 {'H', 'C', 'O'}
Molecule: 1103 29 {'O', 'N', 'C', 'F', 'H'}
Molecule: 1104 29 {'H', 'C', 'O'}
Molecule: 1106 45 {'H', 'C', 'O'}
Molecule: 1107 59 {'H', 'C', 'O'}
Molecule: 1108 33 {'O', 'N', 'S', 'C', 'H'}
Molecule: 1109 42 {'H', 'N', 'C'}
Molecule: 1110 40 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 1111 44 {'H', 'C', 'O'}
Molecule: 1112 58 {'H', 'Cl', 'C', 'O'}
Molecule: 1113 26 {'H', 'N', 'C'}
Molecule: 1114 56 {'H', 'F', 'N', 'C'}
Molecule: 1115 66 {'O', 'N', 'C', 'F', 'H'}
Molecule: 1116 50 {'H', '

Molecule: 1309 48 {'O', 'N', 'C', 'F', 'Cl', 'H'}
Molecule: 1310 48 {'O', 'N', 'S', 'C', 'F', 'H'}
Molecule: 1311 66 {'O', 'N', 'C', 'F', 'H'}
Molecule: 1313 48 {'H', 'C', 'O'}
Molecule: 1314 56 {'O', 'S', 'C', 'F', 'H'}
Molecule: 1315 18 {'H', 'C', 'O'}
Molecule: 1316 77 {'H', 'C', 'O'}
Molecule: 1317 31 {'H', 'N', 'C', 'O'}
Molecule: 1319 55 {'O', 'N', 'S', 'C', 'F', 'H'}
Molecule: 1320 59 {'O', 'N', 'C', 'F', 'Cl', 'H'}
Molecule: 1321 60 {'O', 'N', 'S', 'C', 'H'}
Molecule: 1322 27 {'H', 'Cl', 'N', 'C'}
Molecule: 1323 27 {'H', 'Cl', 'N', 'C'}
Molecule: 1324 49 {'O', 'N', 'C', 'F', 'H'}
Molecule: 1325 50 {'O', 'N', 'C', 'F', 'H'}
Molecule: 1327 30 {'N', 'S', 'C', 'Cl', 'H'}
Molecule: 1329 42 {'H', 'Cl', 'N', 'C'}
Molecule: 1330 45 {'H', 'N', 'C', 'O'}
Molecule: 1331 39 {'O', 'N', 'C', 'Cl', 'H'}
Molecule: 1332 42 {'H', 'N', 'C'}
Molecule: 1335 51 {'H', 'N', 'C', 'O'}
Molecule: 1336 62 {'H', 'N', 'C', 'O'}
Molecule: 1337 69 {'H', 'N', 'C'}
Molecule: 1338 66 {'H', 'C', 'O'}
Molecule: 13

Molecule: 1541 72 {'O', 'N', 'S', 'C', 'H'}
Molecule: 1542 71 {'H', 'N', 'C', 'O'}
Molecule: 1543 57 {'H', 'N', 'C', 'O'}
Molecule: 1544 57 {'H', 'N', 'C', 'O'}
Molecule: 1545 47 {'H', 'N', 'C', 'O'}
Molecule: 1546 28 {'H', 'N', 'C', 'O'}
Molecule: 1547 65 {'O', 'N', 'S', 'C', 'Cl', 'H'}
Molecule: 1548 119 {'H', 'N', 'C', 'O'}
Molecule: 1549 119 {'H', 'N', 'C', 'O'}
Molecule: 1550 56 {'H', 'Cl', 'N', 'C'}
Molecule: 1551 47 {'H', 'C', 'O'}
Molecule: 1552 62 {'H', 'N', 'C', 'O'}
Molecule: 1553 50 {'H', 'N', 'C', 'O'}
Molecule: 1554 60 {'O', 'N', 'C', 'F', 'Cl', 'H'}
Molecule: 1555 43 {'H', 'N', 'C', 'O'}
Molecule: 1556 46 {'H', 'N', 'C', 'O'}
Molecule: 1557 46 {'O', 'N', 'S', 'C', 'H'}
Molecule: 1558 38 {'H', 'N', 'C', 'O'}
Molecule: 1559 13 {'H', 'Cl', 'C', 'O'}
Molecule: 1560 40 {'H', 'C', 'O'}
Molecule: 1561 61 {'O', 'N', 'S', 'C', 'H'}
Molecule: 1562 40 {'O', 'N', 'S', 'C', 'F', 'H'}
Molecule: 1563 26 {'H', 'Cl', 'C', 'O'}
